In [1]:
from google.colab import files
files.upload()  # Upload your kaggle.json file here


Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"adityamanoja","key":"c46c448c223ceb27fbd1363752459eab"}'}

In [2]:
!pip install kaggle


In [3]:
import os
import shutil

# Move the uploaded kaggle.json file to the correct directory
os.makedirs("/root/.kaggle", exist_ok=True)
shutil.move("kaggle.json", "/root/.kaggle/kaggle.json")

# Set the correct permissions
os.chmod("/root/.kaggle/kaggle.json", 0o600)

print("Kaggle API key is successfully configured!")


Kaggle API key is successfully configured!


In [4]:
!pip install tensorflow opencv-python


In [5]:
!kaggle datasets download -d snehilsanyal/construction-site-safety-image-dataset-roboflow


Dataset URL: https://www.kaggle.com/datasets/snehilsanyal/construction-site-safety-image-dataset-roboflow
License(s): Attribution 4.0 International (CC BY 4.0)
 89% 184M/206M [00:01<00:00, 139MB/s]
100% 206M/206M [00:01<00:00, 117MB/s]


In [6]:
# Define the folder where you want to store the dataset
dataset_folder = '/content/construction_site_dataset'

# Create the folder if it doesn't exist
os.makedirs(dataset_folder, exist_ok=True)

print(f"Dataset folder created at {dataset_folder}")


Dataset folder created at /content/construction_site_dataset


In [7]:
import zipfile

# Path to the downloaded dataset zip file
zip_file = '/content/construction-site-safety-image-dataset-roboflow.zip'

# Unzip the dataset into the dataset folder
with zipfile.ZipFile(zip_file, 'r') as zip_ref:
    zip_ref.extractall(dataset_folder)

print(f"Dataset unzipped to {dataset_folder}")


Dataset unzipped to /content/construction_site_dataset


In [8]:
# Path to the css-data folder after unzipping
css_data_dir = os.path.join(dataset_folder, 'css-data')

# List contents of css-data folder
print("Contents of css-data directory:")
print(os.listdir(css_data_dir))

# Now, list the directories inside 'train', 'valid', and 'test'
train_dir = os.path.join(css_data_dir, 'train')
valid_dir = os.path.join(css_data_dir, 'valid')
test_dir = os.path.join(css_data_dir, 'test')

# Check contents of 'train', 'valid', and 'test' directories
print("Train directory contents:", os.listdir(train_dir))
print("Valid directory contents:", os.listdir(valid_dir))
print("Test directory contents:", os.listdir(test_dir))


Contents of css-data directory:
['train', 'README.roboflow.txt', 'valid', 'test', 'README.dataset.txt']
Train directory contents: ['labels', 'images']
Valid directory contents: ['labels', 'images']
Test directory contents: ['labels', 'images']


In [9]:
# Check if images or labels folders exist inside train, valid, and test
train_images_dir = os.path.join(train_dir, 'images')
valid_images_dir = os.path.join(valid_dir, 'images')
test_images_dir = os.path.join(test_dir, 'images')

print("Train images directory contents:", os.listdir(train_images_dir))
print("Valid images directory contents:", os.listdir(valid_images_dir))
print("Test images directory contents:", os.listdir(test_images_dir))


Train images directory contents: ['construction-4-_mp4-19_jpg.rf.9bd721934889a9f63923ffb5285f4d3c.jpg', 'youtube-224_jpg.rf.0e87e77b917fae082722bc8e324092f5.jpg', 'youtube-667_jpg.rf.730ceb4b36b1417c6f952e2f5b0e6406.jpg', 'youtube-550_jpg.rf.2a83bed08e8a623e8261f9c935f70e09.jpg', 'construction-523-_jpg.rf.32a3982052d2961d530c1b49d3fe9a0f.jpg', 'youtube-175_jpg.rf.43dd4680523315f10c35983495275887.jpg', 'youtube-235_jpg.rf.f06d5cefa3f6f6b1ca0b4f707b71f016.jpg', 'youtube-568_jpg.rf.40b08efe635c831ccfeaff4494c412e1.jpg', 'mask-wearing-1632933572733_png_jpg.rf.ef7fc5df6b537c8c859bd7a0e4524a7d.jpg', 'youtube-276_jpg.rf.4db3522d66a80bfdd0afd4e92b7416b6.jpg', 'Bookstore111_jpg.rf.ebe6cd0b7e77dc3cea423b7b3458f67f.jpg', 'image_61_jpg.rf.29cef9c1e3298c6c56c69489d47dd7e1.jpg', 'construction-1062-_jpg.rf.bb9d74532f809eda6343cb0f354fc269.jpg', 'youtube-246_jpg.rf.8809473eb178b062b48fb1ee9b8e5369.jpg', 'construction-118-_jpg.rf.834d29e7ecd402dc301281478710eca5.jpg', '707_jpg.rf.8457eeaff99b07ff4e8ccb

# IMPORT STATEMENTS

In [10]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import zipfile
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import MobileNetV2
import shutil

In [11]:
import os

# Check the contents of 'css-data' directory
os.listdir('/content/construction_site_dataset/css-data')

['train', 'README.roboflow.txt', 'valid', 'test', 'README.dataset.txt']

In [12]:
train_dir = '/content/construction_site_dataset/css-data/train'
valid_dir = '/content/construction_site_dataset/css-data/valid'
test_dir = '/content/construction_site_dataset/css-data/test'


# AUGMENT DATA

In [13]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set up data augmentation for the training set
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,   # Normalize pixel values to [0, 1]
    rotation_range=40,   # Random rotation
    width_shift_range=0.2,  # Random width shift
    height_shift_range=0.2,  # Random height shift
    shear_range=0.2,     # Shear transformation
    zoom_range=0.2,      # Zoom
    horizontal_flip=True,  # Flip images horizontally
    fill_mode='nearest'    # Fill in missing pixels after transformations
)

# Only rescale validation images
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)


# LOAD DATA

In [14]:
# Load training and validation datasets
train_dataset = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Resize to match MobileNetV2's input size
    batch_size=32,
    class_mode='binary'  # 'binary' for binary classification, adjust if needed
)

valid_dataset = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)


Found 2605 images belonging to 2 classes.
Found 114 images belonging to 2 classes.


# BUILD MODEL ARCHITECTURE

In [15]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

# Load MobileNetV2 model pre-trained on ImageNet (without the top classification layer)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the base model layers
base_model.trainable = False

# Build the complete model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Convert 3D feature maps to 1D
    layers.Dense(256, activation='relu'),  # Fully connected layer
    layers.Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d             │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [16]:
# Train the model
#model.fit(train_dataset, epochs=6, validation_data=valid_dataset)


# CONVERT INTO TENSORFLOWLITE

In [17]:
# Save the trained model
model.save('my_model.h5')

# Convert the model to TensorFlow Lite format
import tensorflow as tf

# Convert the model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('my_model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmpiavakby5'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_154')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140440923383136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140440922610624: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140440922611152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140440922605872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140440922608160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140440922616960: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140440922615904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140440922618368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140440922606752: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140440922617136: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1404409226

In [53]:
print(train_dataset.class_indices)


{'images': 0, 'labels': 1}


# TESTING DATA

In [54]:
import os
print(os.listdir("/content"))


['.config', 'my_model.tflite', 'construction-site-safety-image-dataset-roboflow.zip', 'my_model.h5', 'model.tflite', 'construction_site_dataset', 'sample_data']


In [55]:
model.summary()


Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_7           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [56]:
import tensorflow as tf

# Assuming `model` is your trained Keras model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the converted model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)


Saved artifact at '/tmp/tmp6adzuisq'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_959')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140438591735408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140439052938304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438591739984: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140439052940768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140439052943056: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140439052941824: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140439052753152: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140439052755792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140439052751744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140439052753680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  1404390527

#Trial


In [57]:
import os
import zipfile
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import MobileNetV2
import shutil
import numpy as np

In [58]:
dataset_folder = '/content/construction_site_dataset'
css_data_dir = os.path.join(dataset_folder, 'css-data')

# Define the directories for training, validation, and test data
train_dir = os.path.join(css_data_dir, 'train')
valid_dir = os.path.join(css_data_dir, 'valid')
test_dir = os.path.join(css_data_dir, 'test')

# Check if images or labels folders exist inside train, valid, and test
train_images_dir = os.path.join(train_dir, 'images')
valid_images_dir = os.path.join(valid_dir, 'images')
test_images_dir = os.path.join(test_dir, 'images')

# Print the contents of each directory
print("Train images directory contents:", os.listdir(train_images_dir))
print("Valid images directory contents:", os.listdir(valid_images_dir))
print("Test images directory contents:", os.listdir(test_images_dir))


Train images directory contents: ['construction-4-_mp4-19_jpg.rf.9bd721934889a9f63923ffb5285f4d3c.jpg', 'youtube-224_jpg.rf.0e87e77b917fae082722bc8e324092f5.jpg', 'youtube-667_jpg.rf.730ceb4b36b1417c6f952e2f5b0e6406.jpg', 'youtube-550_jpg.rf.2a83bed08e8a623e8261f9c935f70e09.jpg', 'construction-523-_jpg.rf.32a3982052d2961d530c1b49d3fe9a0f.jpg', 'youtube-175_jpg.rf.43dd4680523315f10c35983495275887.jpg', 'youtube-235_jpg.rf.f06d5cefa3f6f6b1ca0b4f707b71f016.jpg', 'youtube-568_jpg.rf.40b08efe635c831ccfeaff4494c412e1.jpg', 'mask-wearing-1632933572733_png_jpg.rf.ef7fc5df6b537c8c859bd7a0e4524a7d.jpg', 'youtube-276_jpg.rf.4db3522d66a80bfdd0afd4e92b7416b6.jpg', 'Bookstore111_jpg.rf.ebe6cd0b7e77dc3cea423b7b3458f67f.jpg', 'image_61_jpg.rf.29cef9c1e3298c6c56c69489d47dd7e1.jpg', 'construction-1062-_jpg.rf.bb9d74532f809eda6343cb0f354fc269.jpg', 'youtube-246_jpg.rf.8809473eb178b062b48fb1ee9b8e5369.jpg', 'construction-118-_jpg.rf.834d29e7ecd402dc301281478710eca5.jpg', '707_jpg.rf.8457eeaff99b07ff4e8ccb

In [59]:
def contrast_stretching(img):
    min_val = tf.reduce_min(img)
    max_val = tf.reduce_max(img)
    stretched_img = (img - min_val) / (max_val - min_val)
    return stretched_img

# Custom ImageDataGenerator that includes contrast stretching
class CustomImageDataGenerator(ImageDataGenerator):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def flow_from_directory(self, directory, *args, **kwargs):
        generator = super().flow_from_directory(directory, *args, **kwargs)
        return self._add_contrast_stretching(generator)

    def _add_contrast_stretching(self, generator):
        while True:
            batch_x, batch_y = next(generator)
            batch_x = np.array([contrast_stretching(img) for img in batch_x])
            yield batch_x, batch_y

In [60]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Training dataset with augmentation
train_datagen = ImageDataGenerator(
    rescale=1.0/255.0,  # Normalize pixel values
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Validation dataset with only rescaling
valid_datagen = ImageDataGenerator(rescale=1.0/255.0)


In [61]:
train_dataset = train_datagen.flow_from_directory(
    '/content/construction_site_dataset/css-data/train',
    target_size=(224,224),  # Use smaller size
    batch_size=32,
    class_mode='binary'
)

valid_dataset = valid_datagen.flow_from_directory(
    '/content/construction_site_dataset/css-data/valid',
    target_size=(224,224),  # Use smaller size
    batch_size=32,
    class_mode='binary'
)


# Load MobileNetV2 base model (without top classification layer)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

Found 2605 images belonging to 2 classes.
Found 114 images belonging to 2 classes.


In [62]:
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras.callbacks import EarlyStopping

# Freezing the base model
base_model.trainable = False

# Build the complete model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),  # Convert 3D feature maps to 1D
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.01)),  # L2 Regularization
    layers.Dense(1, activation='sigmoid')  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()


Model: "sequential_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224 (Functional)    │ (None, 7, 7, 1280)          │       2,257,984 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_8           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 256)                 │         327,936 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │             257 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 2,586,177 (9.87 MB)

 Trainable params: 328,193 (1.25 MB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [63]:
# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

# Train the model with early stopping
history = model.fit(
    train_dataset,  # Training data
    epochs=5,       # Number of epochs
    validation_data=valid_dataset,  # Validation data
    callbacks=[early_stopping]  # Add early stopping callback
)

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


82/82 ━━━━━━━━━━━━━━━━━━━━ 62s 580ms/step - accuracy: 0.9886 - loss: 2.9028 - val_accuracy: 1.0000 - val_loss: 0.5090
Epoch 2/5
82/82 ━━━━━━━━━━━━━━━━━━━━ 35s 398ms/step - accuracy: 1.0000 - loss: 0.3473 - val_accuracy: 1.0000 - val_loss: 0.0970
Epoch 3/5
82/82 ━━━━━━━━━━━━━━━━━━━━ 35s 383ms/step - accuracy: 1.0000 - loss: 0.0670 - val_accuracy: 1.0000 - val_loss: 0.0242
Epoch 4/5
82/82 ━━━━━━━━━━━━━━━━━━━━ 41s 403ms/step - accuracy: 1.0000 - loss: 0.0158 - val_accuracy: 1.0000 - val_loss: 0.0071
Epoch 5/5
82/82 ━━━━━━━━━━━━━━━━━━━━ 35s 383ms/step - accuracy: 1.0000 - loss: 0.0043 - val_accuracy: 1.0000 - val_loss: 0.0026


# **PLOTTING**

In [64]:
# Save the trained model
model.save('my_model_1.h5')

# Convert the model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the TFLite model
with open('my_model_1.tflite', 'wb') as f:
    f.write(tflite_model)

Saved artifact at '/tmp/tmpc2m52grj'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='keras_tensor_1118')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140438537298240: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438591908400: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438591910160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438591906992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438591909456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438591907344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438592456832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438592459648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438592455072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438592458592: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438592

In [68]:
!pip install ethos-u-vela
!!vela --version

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 29.3 MB/s eta 0:00:00


['4.1.0']

In [69]:
import tensorflow as tf
import numpy as np

# Load the existing TensorFlow model
# Make sure you replace '/path/to/your/model' with the correct path to your model
model = tf.keras.models.load_model('/content/my_model_1.h5')  # Replace with your actual model path

# Representative data generator for quantization
# You need to provide a representative dataset for calibration
def representative_data_gen():
    for _ in range(100):  # You can adjust the number of steps based on your dataset
        # Example input shape (replace with actual input shape of your model)
        yield [np.random.rand(1, 224, 224, 3).astype(np.float32)]  # Example shape: (224, 224, 3)

# Convert the model to TensorFlow Lite format with int8 quantization
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Set optimization to reduce model size (int8 quantization)
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Set supported ops to use int8 operations
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]  # Use int8 ops

# Provide the representative dataset for quantization
converter.representative_dataset = representative_data_gen

# Set input and output tensor types to int8
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model to a .tflite file with int8 quantization
tflite_model = converter.convert()

# Save the converted model as a .tflite file
tflite_model_path = '/content/quantized_model_int8.tflite'
with open(tflite_model_path, 'wb') as f:
    f.write(tflite_model)

# Verify that the model was saved correctly
print(f"Model successfully converted to int8 and saved as {tflite_model_path}")


Saved artifact at '/tmp/tmpvkgzdvaf'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input_layer_15')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  140438549735104: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549583072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549585008: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549578496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549580784: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549587648: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549586944: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549589408: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549585888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549587296: TensorSpec(shape=(), dtype=tf.resource, name=None)
  140438549591

/usr/local/lib/python3.10/dist-packages/tensorflow/lite/python/convert.py:983: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(


Model successfully converted to int8 and saved as /content/quantized_model_int8.tflite


In [70]:
!vela quantized_model_int8.tflite --accelerator-config ethos-u55-64


Network summary for quantized_model_int8
Accelerator configuration                Ethos_U55_64
System configuration             Ethos_U55_High_End_Embedded
Memory mode                               Shared_Sram
Accelerator clock                                 500 MHz
Design peak SRAM bandwidth                       3.73 GB/s
Design peak Off-chip Flash bandwidth             0.47 GB/s

Total SRAM used                               1474.22 KiB
Total Off-chip Flash used                     2584.44 KiB

CPU operators = 0 (0.0%)
NPU operators = 67 (100.0%)

Average SRAM bandwidth                           1.33 GB/s
Input   SRAM bandwidth                          12.64 MB/batch
Weight  SRAM bandwidth                          12.00 MB/batch
Output  SRAM bandwidth                           6.65 MB/batch
Total   SRAM bandwidth                          31.48 MB/batch
Total   SRAM bandwidth            per input     31.48 MB/inference (batch size 1)

Average Off-chip Flash bandwidth               

In [71]:
pip install tflite-runtime opencv-python


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 31.4 MB/s eta 0:00:00
